In [ ]:
import re

from nltk.corpus import brown
from nltk.corpus import reuters
from nltk.corpus import words
from nltk.corpus import webtext
from nltk.corpus import nps_chat
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer

In [ ]:
def clean_reviews(export, content_column="Content", clean_content_column="CleanContent"):
    # Conjunto de palavras da lingua inglesa
    english_words_corpus = set(brown.words())\
        .union(set(reuters.words()))\
        .union(set(words.words()))\
        .union(set(webtext.words()))
        
    english_words_corpus2 = set(brown.words())\
        .union(set(reuters.words()))\
        .union(set(words.words()))
        
    # Conjunto de stopwords da língua inglesa
    english_stopwords_corpus = set(stopwords.words("english"))
    # Stemmizador PorterStemmer
    stemmer = PorterStemmer()
    
    # Retorna somente os exports cujo conteúdo não é nulo
    valid_export = export.loc[export[content_column].notnull()]
    
    # Retorna o conteúdo de cada review em formato de lista
    reviews = valid_export[content_column]
    
    # Lista que armazenará as reviews limpas
    clean_reviews = []
    
    for review in reviews:
        # Retira HTML codes que podem estar no texto (ex: &quot;)
        no_html_symbols = re.sub("(\&).+?(\;)", " ", review)
        
        # Retira todos os caracteres que não sejam letras (maiúsculas e minúsculas)
        letters_only = re.sub("[^a-zA-Z]", " ", no_html_symbols)
        
        # Converte todas as letras para lower case
        lower_case = letters_only.lower()
        
        # Separa o texto da review em uma lista de palavras
        words_list = lower_case.split()
        
        # Obtém todas as palavras significativas, ao remover as stopwords
        meaningful_words = [word for word in words_list if word not in english_stopwords_corpus]
        
        # Obtém todas as palavras que pertencem à lingua inglesa
        english_words = [word for word in meaningful_words if word in english_words_corpus]
        
        # Obtém os stems das palavfen ras
        stems = [stemmer.stem(word) for word in english_words]
        
        no_single_letters = [word for word in stems if len(word) > 1]
        
        # Concatena todas as palavras que restaram, separadas por um caractere de espaço.
        clean_review = " ".join(no_single_letters)
        
        if len(clean_review) > 1:
            # Adiciona a review já pré-processada (limpa) à lista de reviews limpas
            clean_reviews.append(clean_review)
        else:
            clean_reviews.append('')
    
    # Insere uma nova coluna no dataframe, contendo o conteúdo dos comentários já pré-processado
    valid_export.insert(loc=len(valid_export.columns), column=clean_content_column, value=clean_reviews)
    
    # Retorna somente os comentários cujo conteúdo não está em branco
    valid_export = valid_export.loc[valid_export[clean_content_column] != '']
    
    # Retorna a lista de exports já pré-processados
    return valid_export